# Basic Prompt Engineering (Korean Language)

## Step 1. Prepare Large Language Model (LLM) and Embedding Model 
---

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [ ]:
import time
import sagemaker, boto3, json
import glob
import os
import pandas as pd
import requests
import json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from ssm import parameter_store

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
pm = parameter_store(aws_region)
pm_restapi_id_key = f"RESTAPI-ID-{aws_region.upper()}"

try:
    RESTAPI_ID = pm.get_params(key=pm_restapi_id_key)
    URL = f'https://{RESTAPI_ID}.execute-api.{aws_region}.amazonaws.com/api/'.replace('"','')
    print("RESTAPI_ID = ", RESTAPI_ID) # YOUR RESTAPI ID
    print("API GATEWAY URL = ", URL)    
except Exception as e:
    print("[ERROR] Please run rag/0_setup.ipynb first!")

In [ ]:
MODEL_NAME = "KULLM-12-8B"

LLM_INFO = {
    "LLAMA2-7B": f"{URL}llm/llama2_7b",
    "FALCON-40B": f"{URL}llm/falcon_40b",    
    "KULLM-12-8B": f"{URL}llm/kkulm_12_8b",
}

LLM_URL = LLM_INFO[MODEL_NAME]
EMB_URL = f"{URL}/emb/kosimcse"

HEADERS = {    
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

if 'falcon_40b' in LLM_URL:
    LLM_RESPONSE_KEY = "generated_text"
else:
    LLM_RESPONSE_KEY = "generation"
    
print (f'MODEL_NAME: {MODEL_NAME}\nLLM_URL: {LLM_URL}')    

In [ ]:
PARAMS = {
    "KULLM-12-8B": {
        "do_sample": False,
        "max_new_tokens": 256,
        "temperature": 0.2,
        "top_p": 0.9,
        "return_full_text": False,
        "repetition_penalty": 1.2,
        "presence_penalty": None,
        "eos_token_id": 2,
    }    
}

<br>

## Step 2. Ask a question to LLM without RAG
---

### Simple prompt engineering

In [ ]:
from lib_ko import Prompter, get_payload
from langchain.llms import AmazonAPIGateway

llm = AmazonAPIGateway(api_url=LLM_URL, headers=HEADERS)
params = PARAMS[MODEL_NAME]
llm.model_kwargs = params

In [ ]:
prompter  = Prompter("kullm")

In [ ]:
%%time

payload = {
    "inputs": "Generative AI는",
    "parameters": params
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

In [ ]:
%%time

payload = {
    "inputs": """Amazon SageMaker의 주요 기능을 간략하게 설명하는 이메일 메시지

여러분 안녕하세요.

다음 사항을 발표합니다.""",
    "parameters": params
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

### More complex prompts: Play the role of AWS SA


In [ ]:
%%time
instruction = """
당신은 AWS에 대한 경험이 풍부한 솔루션 아키텍트입니다. 고객 요구 사항을 분석하여 잘 설계된 솔루션 아키텍처를 만들어 고객에게 제시합니다. 
아키텍처는 상세하고 친절해야 합니다. 다음과 같은 컨텍스트가 주어집니다.
"""

architect_prompt = """
Context:
#요구 사항:
{requirements}
#스케일:
{scale}
#특징:
{features}

AWS의 아키텍처를 기술적으로 자세히 설명하세요.
"""
context = architect_prompt.format(
    requirements="컴퓨터 광고 웹사이트", 
    scale="최대 초당 10,000건의 요청을 처리해야 합니다. 전 세계에서 사용할 수 있어야 합니다. 응답 속도가 빨라야 합니다.",
    features="제품을 설명하는 랜딩 페이지. 회사를 설명하는 소개 페이지."
)

payload = get_payload(instruction, context, params)
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

In [ ]:
%%time

instruction = "다음 글을 알기 쉽게 요약해 주세요."
context = """
대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.
"""

payload = get_payload(instruction, context, params)
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

### Applying LangChain

In [ ]:
llm.model_kwargs = params
print(llm(payload['inputs']))

In [ ]:
from langchain.prompts import PromptTemplate

# First we can define an exposed parameter interface to the format string
prompt = PromptTemplate(
    input_variables=["requirements", "scale", "features"],
    template=architect_prompt,
)

context = architect_prompt.format(
    requirements="자바스크립트로 작성된 외부 웹 애플리케이션, 글로벌 배포",
    scale="분당 평균 500건의 요청, 초당 최대 3000건의 요청까지 이벤트 확장",
    features="모바일 웹사이트, 데스크톱 버전, 자바스크립트"
)
print(context)

In [ ]:
%%time
payload = get_payload(instruction, context, params)
print(llm(payload['inputs']))

In [ ]:
PARAMS = {
    "KULLM-12-8B": {
        "do_sample": False,
        "max_new_tokens": 200,
        "temperature": 0.4,
        "top_p": 0.95,
        "return_full_text": False,
        "repetition_penalty": 1.5,
        "presence_penalty": None,
        "eos_token_id": 2,
    }    
}

In [ ]:
topic_recommender_prompt = "{topic}에 대한 블로그 게시물에 작성할 세부 주제를 {number}개만큼 나열합니다. 머신러닝 전처리 방법이나 모델 훈련 방법이 포함되어야 합니다."

recommend_topic_prompt = PromptTemplate(
    template=topic_recommender_prompt,
    input_variables=['topic', 'number'],
    model_kwargs=PARAMS[MODEL_NAME]
)

results = llm(recommend_topic_prompt.format(topic="머신 러닝", number=5))
print(results)

In [ ]:
instruction = "다음 질문에 답변해 주세요."
question = "Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스는 무엇인가요?"
payload = get_payload(instruction, question, params)

response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])
#payload = get_payload(instruction, context, params)

### With Context
- 추가 컨텍스트 or few-shot 제공

In [ ]:
instruction = "위 컨텍스트 내용에 기반해서 질문에 답변해 주세요."
context = "관리형 스팟 훈련은 Amazon SageMaker에서 지원되는 모든 인스턴스에서 사용할 수 있고, 현재 Amazon SageMaker를 사용할 수 있는 모든 AWS 리전에서 지원됩니다."

question = "Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스는 무엇인가요?"
prompt = f"{context}\n\n{question}"

In [ ]:
payload = get_payload(instruction, question, params)

response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

### Apply LangChain


In [ ]:
result = llm(prompt)
print(result)